In [24]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from urllib.parse import urlparse
from tldextract import extract
from numba import jit

import wordsegment as ws
ws.load()

from code.util import *
from code.preprocessor import *
from code.gofaster import *

gf = GoFaster(11, 22)

N0 = 45000
N1 = 15000
VOCAB = 10000
CVECT = 100
WVECT = 20
UCHARS = 139
MI = 30

# Sampling

In [ ]:
url0 = pd.read_csv("data/raw/url_0.csv", skiprows=0).sample(
    n=N0,
    random_state=11
).reset_index(drop=True)

url1 = pd.read_csv("data/raw/url_1.csv", skiprows=0).sample(
    n=N1,
    random_state=11
).reset_index(drop=True)

In [ ]:
url0["target"] = pd.Series(np.zeros(N0).astype(np.int32))
url1["target"] = pd.Series(np.ones(N1).astype(np.int32))

In [ ]:
url = pd.concat(
    [
        url0[["url", "target"]],
        url1[["url", "target"]]
    ]
)

In [ ]:
save(url, "data/url_45_15.pkl")

# X, Y

In [ ]:
x0 = url.url
y0 = url.target

# Target Onehot

In [ ]:
y_onehot = to_categorical(y0, dtype=np.int32)
np.save("data/xy/y_onehot_45_15.npy", y_onehot)

# Vectorizers

In [ ]:
tk = Tokenizer(
    num_words=VOCAB,
    filters='!"#$%&()*+,-.:;<=>?@[\\]^_`{|}~',
    split="/"
)

In [ ]:
tk.fit_on_texts(x0)
x_word = pad_sequences(tk.texts_to_sequences(x0), maxlen=WVECT)
x_word.shape

In [ ]:
np.save("data/xy/x_word_45_15_10000.npy", x_word)

# Character Level Vectorizer

In [2]:
url = load("data/url_45_15.pkl")

In [3]:
x_char = np.vstack(url.url.apply(char_level_encoder, args=(CVECT,)))

In [4]:
x_char

array([[104, 116, 116, ...,   0,   0,   0],
       [104, 116, 116, ...,   0,   0,   0],
       [104, 116, 116, ...,   0,   0,   0],
       ...,
       [104, 116, 116, ..., 117, 115, 116],
       [104, 116, 116, ...,   0,   0,   0],
       [104, 116, 116, ...,   0,   0,   0]])

In [5]:
x_char.shape

(60000, 100)

## Onehot

In [6]:
unique_chars = np.unique(x_char).astype(int)
len(unique_chars)

140

In [7]:
x_char_onehot = char_onehot(x_char, unique_chars, UCHARS)
x_char_onehot.shape

(60000, 13900)

In [8]:
np.savez_compressed("data/xy/x_char_onehot_45_15.npz", x_char_onehot)

# Feature Engineering

In [51]:
data = load("data/url_45_15.pkl")

In [52]:
mi_words = load("data/mi_100.pkl")
mi_words = list(filter(lambda word: True if len(word) >= 3 else False, words))

suspicious_words = ["confirm", "account", "banking", "secure", "ebayisapi", "webscr",
           "login", "signin", "exe", "zip"]

companies = ["ebay", "paypal", "bankofscotland", "volksbank", "wellsfargo",
            "bankofamerica", "privatebanking", "hsbc", "chase", "amazon",
            "banamex", "barclays"]

keywords = list(set(mi_words + suspicious_words + companies))

In [53]:
suspicious_tlds = ['country','stream','download','xin','gdn','racing', 
                   'jetzt','win','bid','vip', 'ren', 'kim', 'loan',
                   'mom', 'party', 'review', 'trade', 'date', 'wang',
                   'accountants', 'zip','cricket','link','work','gq',
                   'science','tk', 'world', 'fit', 'work' 'ryukyu',
                   'life', 'cloud', 'desi', 'okinawa', 'ooo','men',
                   'click', 'loan', 'top', 'cf', 'ml', 'ga']

In [54]:
@jit
def non_ascii(url):
    n = 0
    
    for c in url:
        if ord(c) > 127:
            n += 1
            
    return n

In [55]:
def contains_ip(url):
    p = "\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"
    result = re.findall(p, url)
    return 1 if len(result) > 0 else 0

In [56]:
data["tokens"] = data.url.apply(tokenize)
data["len_url"] = data.url.apply(len)
data["len_tok"] = data.tokens.apply(len)
data["slash"] = data.url.apply(lambda url: url.count("/"))
data["dbl_slash"] = data.url.apply(lambda url: url.count("//"))
data["dot"] = data.url.apply(lambda url: url.count("."))
data["at"] = data.url.apply(lambda url: url.count("@"))
data["at2"] = data.url.apply(lambda url: url.count("%40"))
data["dash"] = data.url.apply(lambda url: url.count("-"))
data["qmark"] = data.url.apply(lambda url: url.count("?"))
data["amp"] = data.url.apply(lambda url: url.count("&"))
data["hash"] = data.url.apply(lambda url: url.count("#"))
data["perc"] = data.url.apply(lambda url: url.count("%"))
data["eq"] = data.url.apply(lambda url: url.count("="))
data["colon"] = data.url.apply(lambda url: url.count(":"))
data["scolon"] = data.url.apply(lambda url: url.count(";"))
data["scheme"] = data.url.apply(lambda url: urlparse(url).scheme)
data["netloc"] = data.url.apply(lambda url: urlparse(url).netloc)
data["tld"] = data.netloc.apply(lambda netloc: extract(netloc).suffix)
data["suspicious"] = data.tld.apply(lambda tld: 1 if tld in suspicious_tlds else 0)
data["non_ascii"] = data.url.apply(non_ascii)
data["contains_ip"] = data.url.apply(contains_ip)
data["kw_count"] = data.tokens.apply(lambda tokens: len(set(tokens) & set(keywords)))

for i in range(len(keywords)):
    data["kw%d" % (i)] = data.tokens.apply(lambda tokens: 1 if keywords[i] in tokens else 0)
    
dummies = pd.get_dummies(data.scheme, drop_first=True)
data = pd.concat([data, dummies], axis=1)

In [57]:
drop = [
    "target",
    "url",
    "tokens",
    "scheme",
    "netloc",
    "tld"
]

data = data.drop(columns=drop, axis=1)

In [58]:
data.values.shape

(60000, 118)

In [59]:
np.save("data/xy/x_feat_45_15.npy", data.values)

# Segments

In [ ]:
data = load("data/url_45_15.pkl")

In [ ]:
def segment(tokens):
    segments = []
    
    for token in tokens:
        segments.append(" ".join(ws.segment(token)))
        
    return " ".join(segments)

In [ ]:
def worker(df):
    df["segments"] = df.tokens.apply(segment)
    return df

In [ ]:
data["tokens"] = data.url.apply(tokenize)

In [ ]:
data = gf.parallelize(data, worker)

In [ ]:
data[["tokens", "segments"]].head(30)

In [ ]:
tk = Tokenizer(
    num_words=VOCAB
)

In [ ]:
tk.fit_on_texts(data.segments)
x_segments = pad_sequences(tk.texts_to_sequences(data.segments), maxlen=WVECT)
x_segments.shape

In [ ]:
np.save("data/xy/x_segments_45_15.npy", x_segments)